In [1]:
from EAC import site
import os 
import pandas as pd
import re
import numpy as np
import sqlite3

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
training = site.training()

conn = sqlite3.connect('training.db')

In [18]:
conn.execute('DROP TABLE movements')
conn.commit()

OperationalError: no such table: movements

In [28]:
conn.execute(create_movement_table_q)

In [ ]:
create_meso_q = '''
CREATE TABLE upcomingTraining (
week int,
session char(20),
movement char(20),
method char(20),
sets int,
reps int,
weight float(4),
progNotes char(140),
targetReps int
)
'''

In [152]:
#each movement/lift is the pk
create_movement_table_q = '''
CREATE TABLE movements (
movement CHAR(40) PRIMARY KEY not null,
split CHAR(10),
actualMax int,
repMaxCalc float(4),
veloEquation CHAR(30),
repMax10 int,
repMax9 int,
repMax8 int,
repMax7 int,
repMax6 int,
repMax5 int,
repMax4 int,
repMax3 int,
repMax2 int,
kilos binary
);

'''

In [90]:
#methodID is a amalgamation of method and week has 8 columns
create_program_builder_q = '''
CREATE TABLE programBuilder (
methodID char(30) PRIMARY KEY not null,
method char(20),
week int,
sets int,
reps int,
weight float(4),
notes char(140),
targetReps int
);
'''

In [ ]:
#sessionID  = date_exercise
create_history_q = '''
CREATE TABLE history(
sessionID PRIMARY KEY not null,
sessionDate date,
session char(30),
movement CHAR(40),
method CHAR(20),
setsCompleted int,
repsCompleted int,
weightUsed int,
amrapTotal int,
newMax binary,
notes CHAR(200),
RepCalcM int);
'''

In [ ]:
#veloID = date_exercise_set_rep
create_velo_q='''
CREATE TABLE velo (
movement char(40),
sessionDate date,
weight_kg float(4),
weight_lb float(4),
set_ct int,
rep int,
ecc_con char(4),
MPV_ms float(4),
peak_velo float(4),
rom_cm float(2),
rom_in float(2),
watts int,
est_max_kg float(4),
est_max_lb float(4)


)
'''

In [ ]:
create_rpe_q = '''
CREATE TABLE rpe (
RPE float(1) PRIMARY KEY not null,
rep1 float(4),
rep2 float(4),
rep3 float(4),
rep4 float(4),
rep5 float(4),
rep6 float(4),
rep7 float(4),
rep8 float(4),
rep9 float(4),
rep10 float(4),
rep11 float(4),
rep12 float(4),
rep13 float(4),
rep14 float(4),
rep15 float(4),
rep16 float(4)
);'''

In [ ]:

conn.execute(create_program_builder_q)
# conn.execute(create_meso_q)
# conn.execute(create_rpe_q)
# conn.execute(create_history_q)
# conn.execute(create_movement_table_q)
# conn.execute(create_velo_q)
conn.commit()


### Data Loading

In [275]:
def create_meso_cycle(path,connection):
    df = pd.DataFrame(columns = ['movement','method','week','sets','reps','weight','notes','targetReps'])
    prgbld = pd.read_sql('''
    select methodID, lower(method) as method, week, sets,reps,weight,notes,targetReps
    from programBuilder
    ''',conn)
    mesoDF = pd.read_csv(dataPath+trainingFile).fillna(0)
    maxes = pd.read_sql('''
    select movement,actualMax
    from movements
    ''',conn)
    v = df.columns[-5:].values
    r=0
    for sesh in mesoDF.session.unique():
        d = mesoDF[mesoDF.session == sesh]
        for movement in d.exercise.unique():
            endweek = d[d.exercise==movement].endWeek.values[0]
            mod = d[d.exercise==movement].modality.values[0]
            cyclelength = prgbld[prgbld.method==mod].week.max()
            setAdjust = d[d.exercise==movement].SetAdjust.values[0]
            for i in range(endweek):
                lkup = i+1 if cyclelength==endweek else (i+1) % cyclelength
                lkup = cyclelength if lkup == 0 else lkup
                l = [movement,mod,i+1]+prgbld[(prgbld.method==mod) & (prgbld.week==lkup)].filter(v).values.tolist()[0]
                df.loc[len(df)] = l
                df.at[r,'sets'] = df.at[r,'sets'] + setAdjust
                r+=1
    final = df[['movement','method']].drop_duplicates().reset_index(drop=True)
    final.movement = final.movement.str.lower()
    final = final.merge(maxes,how='left',on='movement')
    for wk in df.week.unique():
        final['sets{}'.format(wk)] = df[df.week==wk].sets.values
        final['reps{}'.format(wk)] = df[df.week==wk].reps.values
        final['weight{}'.format(wk)] = df[df.week==wk].weight.values
        final['targetReps{}'.format(wk)] = df[df.week==wk].targetReps.values
    print('upload has been sent to InitialData')
    final.to_csv('./InitialData/meso_upload.csv')
    return final
def add_data(path,table,connection,insert=False):
    if insert:
        cols = pd.read_sql('select * from {} limit 1'.format(table),conn).columns
        qs = (len(cols)-1)*'?,'+'?'
        connection.execute('''INSERT into {t} values ({row})'''.format(t=table,row=qs),path)
        i = 1
    else:
        with open(path,'r') as d:
            for i,row in enumerate(d):
                if i==0:
                    continue
                else:
                    row = row.replace('\n','')
                    qs = (len(row.split(','))-1)*'?,'+'?'
                    connection.execute('''INSERT into {t} values ({row})'''.format(t=table,row=qs),row.split(','))
    
    connection.commit()
    print('{} has been updated with {} rows of data'.format(table,i))
    


In [264]:
def update_movements_table(training_path,conn):
    '''
    Will take a csv of the training file, will find differences and update the database
    '''
    training = pd.read_csv('./InitialData/training - TrainingCopy.csv',skiprows=1)
    training.movement = training.movement.str.lower()
    df = training.filter(['movement']+[col for col in training.columns if col.find('wrk')>-1])
    df = df.dropna().set_index('movement')
    mvmts = '","'.join(df.index.str.lower().values)
    mvmt = pd.read_sql('''select * from movements where movement in ("{}")'''.format(mvmts),conn)
    ct=0
    for lft in df.index.values:
    
        lft = lft.lower()
        print(lft)
        try:
            if lft not in mvmt.movement.str.lower().values:
                print('\tAdding into movements table'.format(lft))
                d = [lft,None,df.loc[lft]['wrk_10']] + [np.nan] * (len(mvmt.columns) - 3)
                add_data(d,'movements',conn,True)
                ct+=1
            else:
                print('\tHave {} in database table'.format(lft))
        except:
            print('\t{} NOT ADDED'.format(lft.upper()))
        try:
            if df.loc[lft]['wrk_10'] != mvmt[mvmt.movement==lft]['actualMax'].values[0]:
                print('\tNew working max')
                conn.execute('''UPDATE movements
                        SET actualMax = {0} where movement = "{1}" '''.format(df.loc[lft]['wrk_10'],lft))
            ct+=1
        except:
            print('\tMax same as prior meso') 
        conn.commit()
    print('updated movements table with {} edits'.format(ct))

In [148]:
lft  = 'tsunami squat'
if tdf.loc[lft]['wrk_10'] != mvmt[mvmt.movement==lft]['actualMax'].values[0]:
    print('\tNew working max')
    conn.execute('''UPDATE movements
            SET actualMax = {0} where movement = "{1}" '''.format(tdf.loc[lft]['wrk_10'],lft))

	New working max


In [265]:
update_movements_table('./InitialData/training - TrainingCopy.csv',conn)

slingshot
	Adding into movements table
movements has been updated with 1 rows of data
	Max same as prior meso
bandbell incline
	Adding into movements table
movements has been updated with 1 rows of data
	Max same as prior meso
hatfield
	Have hatfield in database table
	New working max
tsnumai squat
	Adding into movements table
movements has been updated with 1 rows of data
	Max same as prior meso
trap
	Have trap in database table
	New working max
meadows
	Have meadows in database table
pinpress
	Have pinpress in database table
	New working max
standing mil
	Adding into movements table
movements has been updated with 1 rows of data
	Max same as prior meso
tempo
	Adding into movements table
movements has been updated with 1 rows of data
	Max same as prior meso
good morning
	Adding into movements table
movements has been updated with 1 rows of data
	Max same as prior meso
pendlay row
	Have pendlay row in database table
	New working max
seal row
	Have seal row in database table
	New workin

## Process in the future is 
1. run update_movements_table()
2. create meso
3. load into our google sheet
4. when done, upload training file in update movements table

### next steps:
we have the setup to find if lifts only went up during the cycle - .is_monotonic_increasing
can grab the new max from wrk10
\

In [155]:
import os
dataPath = './InitialData/'

In [156]:
pgFile = 'MesoLoad_06022023 - ProgramBuilder.csv'
moveFile = 'MesoLoad_06022023 - lifts.csv'
rpeFile='MesoLoad_06022023 - RPE.csv'
trainingFile = 'MesoLoad_052522024 - MesoLoad_18_PPL.csv'

In [127]:
conn.execute('DROP TABLE programBuilder')
conn.execute(create_program_builder_q)
conn.commit()

In [279]:
conn.execute('DROP TABLE movements')
conn.commit()

In [280]:
conn.execute(create_movement_table_q)
conn.commit()
add_data(dataPath + moveFile,'movements',conn)

movements has been updated with 78 rows of data
